In [1]:
nb_folds = 5
seed = 2024

input_dir = "input/"
labels_dir = input_dir + "station_data"
satelite_dir = input_dir + "satelite_data"

!echo $labels_dir
!ls -alh $labels_dir | head

!echo $satelite_dir
!ls -alh $satelite_dir | tail

input/station_data
total 4.0M
drwxrwxr-x 2 moto moto 4.0K Jan 26 20:41 .
drwxrwxr-x 4 moto moto 4.0K Feb 12 11:15 ..
-rw-rw-r-- 1 moto moto 349K Aug 25 01:19 Atlantic_City_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 329K Aug 25 01:19 Baltimore_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 315K Aug 25 01:19 Eastport_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 321K Aug 25 01:19 Fort_Pulaski_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 291K Aug 25 01:19 Lewes_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 335K Aug 25 01:19 New_London_1993_2013_training_data.csv
-rw-rw-r-- 1 moto moto 313K Aug 25 01:19 Newport_1993_2013_training_data.csv
input/satelite_data
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131222_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131223_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:41 dt_ena_20131224_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21:42 dt_ena_20131225_vDT2021.nc
-rw-rw-r-- 1 moto moto 241K Aug 25 21

In [2]:
# !pip install netCDF4
import netCDF4
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [3]:
import numpy as np
import pandas as pd
import pickle
import glob

from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
np.bool = bool

from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm


# Mine
from Isolation import IsolationModel, GcForestModel

import utils
utils.seed_basic(seed)

!python --version

Python 3.9.21


In [4]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_curve, auc

In [5]:
def get_best_threshold(y_pred, y_val, step_range=0.001, verbose=False):
    # Get the probability values for the positive class
    y_prob = y_pred[:, 1] if (len(y_pred.shape) >= 2) else y_pred
    y_test = y_val[:]
    
    # Initialize the variables to store the best threshold and F1 score
    best_threshold = 0
    best_f1 = 0
     
    # Loop through the possible threshold values from 0 to 1 with a step of 0.01
    for threshold in np.arange(0, 1, step_range):
        # Apply the threshold to the probability values to get the predicted labels
        y_pred_bin = (y_prob >= threshold).astype(int)
        if (y_pred_bin.mean() > 0) and (y_pred_bin.mean() < 1):
            # Calculate the F1 score for the current threshold
            f1 = f1_score(y_test, y_pred_bin)
            # Update the best threshold and F1 score if the current F1 score is higher
            if f1 > best_f1:
                best_threshold = threshold
                best_f1 = f1

    auc = -1
    if (y_val.mean() > 0) and (y_val.mean() < 1):
        auc = roc_auc_score(y_val, y_prob)
    
    if verbose:
        # Print the best threshold and F1 score
        print(f"The optimal threshold is {best_threshold:.2f}")
        print(f"The F1 score using the optimal threshold is {best_f1:.3f}")

    return float(best_threshold), float(round(best_f1, 3)), float(round(auc, 3))

In [6]:
filenames = glob.glob(f"{labels_dir}/*.csv")
df_full = []
for filename in filenames:
    if "merged" in filename:
        continue
    print(filename)
    df_run = pd.read_csv(filename)
    df_full.append(df_run)

print(len(df_full))
df_full = pd.concat(df_full).reset_index(drop=True)

first_date = '1993-01-01'
first_year = int(first_date[:4])

print(df_full.groupby(["location"])["anomaly"].agg(["count", "mean", "sum"]))
df_full["fold"] = df_full["t"].apply(lambda t: (int(t[:4]) - first_year) % nb_folds)
print(df_full.groupby(["fold"])["anomaly"].agg(["count", "mean"]))
df_full.tail()

input/station_data/Fort_Pulaski_1993_2013_training_data.csv
input/station_data/Eastport_1993_2013_training_data.csv
input/station_data/Sewells_Point_1993_2013_training_data.csv
input/station_data/Lewes_1993_2013_training_data.csv
input/station_data/The_Battery_1993_2013_training_data.csv
input/station_data/New_London_1993_2013_training_data.csv
input/station_data/Portland_1993_2013_training_data.csv
input/station_data/Baltimore_1993_2013_training_data.csv
input/station_data/Newport_1993_2013_training_data.csv
input/station_data/Washington_1993_2013_training_data.csv
input/station_data/Atlantic_City_1993_2013_training_data.csv
input/station_data/Sandy_Hook_1993_2013_training_data.csv
12
               count      mean  sum
location                           
Atlantic City   7429  0.043478  323
Baltimore       7649  0.023271  178
Eastport        7495  0.062041  465
Fort Pulaski    7628  0.014552  111
Lewes           7639  0.041367  316
New London      7607  0.004470   34
Newport         7

,t,anomaly,location,latitude,longitude,fold
90443,2013-12-27,0,Sandy Hook,40.466944,-74.009444,0
90444,2013-12-28,0,Sandy Hook,40.466944,-74.009444,0
90445,2013-12-29,0,Sandy Hook,40.466944,-74.009444,0
90446,2013-12-30,0,Sandy Hook,40.466944,-74.009444,0
90447,2013-12-31,0,Sandy Hook,40.466944,-74.009444,0


In [7]:
df_full.groupby("t")["location"].count().sort_values()

t
2002-02-28     9
2004-01-25     9
2004-01-24     9
2004-01-16     9
2004-02-01     9
              ..
2000-02-17    12
2000-02-16    12
2000-02-15    12
2000-02-25    12
2013-12-31    12
Name: location, Length: 7670, dtype: int64

In [8]:
df_full[df_full["t"] == "2007-04-19"].sort_values("location")

,t,anomaly,location,latitude,longitude,fold
80449,2007-04-19,1,Atlantic City,39.356667,-74.418053,4
58113,2007-04-19,1,Baltimore,39.266944,-76.579444,4
12692,2007-04-19,1,Eastport,44.904598,-66.982903,4
5179,2007-04-19,1,Fort Pulaski,32.036700,-80.901700,4
27973,2007-04-19,1,Lewes,38.782780,-75.119164,4
42843,2007-04-19,1,New London,41.361401,-72.089996,4
65738,2007-04-19,0,Newport,41.504333,-71.326139,4
50473,2007-04-19,1,Portland,43.658060,-70.244170,4
88048,2007-04-19,1,Sandy Hook,40.466944,-74.009444,4
20332,2007-04-19,0,Sewells Point,36.946701,-76.330002,4


In [9]:
df_full["year"] = df_full["t"].apply(lambda t: int(t[:4]))
df_full.groupby("year")["anomaly"].agg(["count", "mean", "sum"]).sort_values("mean")                         

,count,mean,sum
year,,,
1994,4330,0.012933,56
2002,4255,0.013161,56
2001,4205,0.014744,62
2004,3934,0.015506,61
1999,4260,0.015962,68
2000,4352,0.016544,72
1995,4367,0.019006,83
2008,4366,0.023591,103
2007,4364,0.026123,114


In [10]:
locations = [
    "Date",
    "Average_SLA",
    "Atlantic City",
    "Baltimore",
    "Eastport",
    "Fort Pulaski",
    "Lewes",
    "New London",
    "Newport",
    "Portland",
    "Sandy Hook",
    "Sewells Point",
    "The Battery",
    "Washington",
][2:]
print(locations)

['Atlantic City', 'Baltimore', 'Eastport', 'Fort Pulaski', 'Lewes', 'New London', 'Newport', 'Portland', 'Sandy Hook', 'Sewells Point', 'The Battery', 'Washington']


In [11]:
filenames = glob.glob(f"{satelite_dir}/*.nc")
print(len(filenames))

dict_sla = {}
for filename in tqdm(filenames):
    date_str = filename.split("/")[-1].split(".")[0].split("_")[2] # dt_ena_19930101_vDT2021.nc"
    if len(date_str) != 8:
        continue
    formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"

    nc_filename = filename
    
    dataset = netCDF4.Dataset(nc_filename, mode="r")
    sla = dataset.variables["sla"][:]
    sla = np.ma.getdata(sla[0, :, :].filled(0))
    dict_sla[formatted_date] = sla

    dataset.close()
len(dict_sla)

7302


100%|██████████████████████████████████████| 7302/7302 [00:54<00:00, 134.54it/s]


7302

In [12]:
dates = list(dict_sla.keys())
dates.sort()
print(len(dates), dates[:3], dates[-3:])

7302 ['1993-01-01', '1993-01-02', '1993-01-03'] ['2013-12-29', '2013-12-30', '2013-12-31']


In [13]:
cities_coordinates = {
    "Atlantic City": {"latitude": 39.3643, "longitude": -74.4229},
    "Baltimore": {"latitude": 39.2904, "longitude": -76.6122},
    "Eastport": {"latitude": 44.9034, "longitude": -66.9895},
    "Fort Pulaski": {"latitude": 32.0317, "longitude": -80.8970},
    "Lewes": {"latitude": 38.7746, "longitude": -75.1394},
    "New London": {"latitude": 41.3557, "longitude": -72.0995},
    "Newport": {"latitude": 41.4901, "longitude": -71.3128},
    "Portland": {"latitude": 43.6591, "longitude": -70.2568},
    "Sandy Hook": {"latitude": 40.4649, "longitude": -74.0054},
    "Sewells Point": {"latitude": 36.9463, "longitude": -76.3297},
    "The Battery": {"latitude": 40.7033, "longitude": -74.0170},
    "Washington": {"latitude": 38.9072, "longitude": -77.0369},
}

# 100 160
cities_indexes = {
    'Atlantic City': {'latitude': (55, 60), 'longitude': (100, 105)},
    'Baltimore': {'latitude': (55, 60), 'longitude': (91, 96)},
    'Eastport': {'latitude': (77, 82), 'longitude': (130, 135)},
    'Fort Pulaski': {'latitude': (26, 31), 'longitude': (74, 79)},
    'Lewes': {'latitude': (53, 58), 'longitude': (97, 102)},
    'New London': {'latitude': (63, 68), 'longitude': (109, 114)},
    'Newport': {'latitude': (63, 68), 'longitude': (112, 117)},
    'Portland': {'latitude': (72, 77), 'longitude': (116, 121)},
    'Sandy Hook': {'latitude': (59, 64), 'longitude': (101, 106)},
    'Sewells Point': {'latitude': (45, 50), 'longitude': (92, 97)},
    'The Battery': {'latitude': (60, 65), 'longitude': (101, 106)},
    'Washington': {'latitude': (53, 58), 'longitude': (89, 94)}
}

def get_features_row(row, verbose=False, margin = 3):
    sla = dict_sla[row["t"]]
    indexes = cities_indexes[row["location"]]
    latitude = indexes["latitude"]
    longitude = indexes["longitude"]

    sla_row = sla[latitude[0] - margin : latitude[1] + margin, longitude[0] - margin : longitude[1] + margin]
    if verbose:
        print(sla.shape, sla_row.shape)
        print(sla.shape)

    return sla_row

get_features_row(df_full.iloc[0], verbose=True)

(100, 160) (11, 11)
(100, 160)


array([[ 0.    ,  0.    ,  0.    , -0.0195, -0.0245, -0.0204, -0.0118,
         0.    ,  0.0127,  0.0235,  0.0292],
       [ 0.    ,  0.    , -0.0173, -0.0157, -0.0197, -0.0251, -0.0305,
        -0.025 , -0.0001,  0.029 ,  0.045 ],
       [ 0.    ,  0.    ,  0.    , -0.019 , -0.0162, -0.0268, -0.0362,
        -0.0282, -0.0114,  0.016 ,  0.038 ],
       [ 0.    ,  0.    ,  0.    , -0.0142, -0.0128, -0.0228, -0.036 ,
        -0.0271, -0.0184, -0.0039, -0.0034],
       [ 0.    ,  0.    ,  0.    ,  0.    , -0.0039, -0.0206, -0.0248,
        -0.0315, -0.0382, -0.0372, -0.033 ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , -0.0259, -0.0245,
        -0.0213, -0.0208, -0.0418, -0.0426],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , -0.0031,  0.0283,
        -0.0062, -0.0147, -0.0205, -0.0213],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        -0.0028, -0.0072, -0.0016,  0.0103],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
 

In [14]:
def sampling(df, target_ratio=0.02, target_col="anomaly"):
    if df[target_col].mean() >= target_ratio:
        return df.copy()
    
    df_pos = df[df[target_col] == 1].copy()
    df_neg = df[df[target_col] != 1].copy().sample(frac=1, random_state=seed).reset_index(drop=True)

    nb_neg = int( (df_pos.shape[0])*(1-target_ratio) / target_ratio)
    df_com = pd.concat([df_pos, df_neg.head(nb_neg)]).sample(frac=1, random_state=seed).reset_index(drop=True)
    return df_com

df_com = sampling(df_full)
print(df_com["anomaly"].value_counts())
df_com["anomaly"].value_counts(normalize=True)

anomaly
0    87950
1     2498
Name: count, dtype: int64


anomaly
0    0.972382
1    0.027618
Name: proportion, dtype: float64

In [15]:
city = locations[3]
fold = 0
dict_models = {}
target_col = "anomaly"
ignored_cities = ["Newport"]

# if True:
run_folds = [-1] # list(range(nb_folds)) + [-1]
print("run_folds:", run_folds)
run_margins = [3, 5, 7, 10, 15, 20] # list(range(nb_folds)) + [-1]
print("run_margins:", run_margins)

run_max_samples = [100, 500, 1000, 3000] # list(range(nb_folds)) + [-1]
print("run_max_samples:", run_max_samples)

for city in locations:
    print("="*40)
    dict_models[city] = []
        
    df_folds = df_full[df_full["location"] == city]
    df_folds = df_folds[df_folds["t"].isin(dates)].sort_values("t").reset_index(drop=True)
    print(city, df_folds.shape, df_folds[target_col].mean())
    
    for fold in run_folds:
        df_fold_train = df_folds[df_folds["fold"] != fold].reset_index(drop=True)
        if fold >= 0:
            df_fold_val = df_folds[df_folds["fold"] == fold].reset_index(drop=True)
        else:
            df_fold_val = df_fold_train.copy()

        # Only sampling the train set
        # df_fold_train = sampling(df_fold_train).sort_values("t").reset_index(drop=True)

        for margin in run_margins:
            print("="*40)
            print( f"Processing margin {margin} city {city}  fold {fold} ... ")
        
            X_train = []
            for index, row in tqdm(df_fold_train.iterrows()):
                img = get_features_row(row, margin=margin)
                X_train.append(img.flatten())
            X_train = np.stack(X_train)
            
            X_val = []
            for index, row in tqdm(df_fold_val.iterrows()):
                img = get_features_row(row, margin=margin)
                X_val.append(img.flatten())
            X_val = np.stack(X_val)
            
            y_train = np.array(df_fold_train[target_col])
            y_val = np.array(df_fold_val[target_col])
            
            print(X_train.shape, X_val.shape, y_train.mean(), y_val.mean())

            for max_samples in run_max_samples:
                        
            # model = CascadeForestClassifier(random_state=1)
            # model = gcForest(shape_1X=[8,8], window=[4,6], tolerance=0.0, min_samples_mgs=10, min_samples_cascade=7)
            # model.fit(X_train, y_train) # Be careful when fold==-1, train == val
                model = IsolationModel(max_samples=max_samples, random_state=seed)
                model.fit(X_train)
                            
                y_pred = model.predict_proba(X_val)[:,1]
                best_threshold, best_f1, auc = get_best_threshold(y_pred, y_val)

                y_pred_bin = (y_pred >= best_threshold)
                acc = accuracy_score(y_val, y_pred_bin)
                
                if (y_val.mean() == 0) or (y_val.mean() == 1):
                    auc = -1
                    f1 = -1
                else:
                    auc = float(roc_auc_score(y_val, y_pred))
                    f1 = float(f1_score(y_val, y_pred_bin, pos_label=1))
        
                print(f"F1: {f1}, ACC: {acc}, AUC: {auc}")
                
                dict_models[city].append( {
                    "model": model,
                    "params": {
                        "fold": fold,
                        "margin": margin,
                        "max_samples": max_samples,
                        "best_threshold": best_threshold,

                        "acc": acc,
                        "auc": auc,
                        "best_f1": f1,
                    },
                    "y_pred": y_pred,
                    "y_val": y_val,
                    "df_val": df_fold_val
                }
                )
                print("="*40)
        
len(dict_models)

run_folds: [-1]
run_margins: [3, 5, 7, 10, 15, 20]
run_max_samples: [100, 500, 1000, 3000]
Atlantic City (7063, 7) 0.04289961772617868
Processing margin 3 city Atlantic City  fold -1 ... 


7063it [00:00, 10222.87it/s]
7063it [00:00, 10231.12it/s]


(7063, 121) (7063, 121) 0.04289961772617868 0.04289961772617868
F1: 0.1358936484490399, ACC: 0.91717400538015, AUC: 0.5514299802761342
F1: 0.11505922165820642, ACC: 0.9259521449808863, AUC: 0.5375485773429414
F1: 0.10465116279069768, ACC: 0.9345887016848364, AUC: 0.5309283886968579
F1: 0.10276679841897234, ACC: 0.935721364859125, AUC: 0.5299097291385942
Processing margin 5 city Atlantic City  fold -1 ... 


7063it [00:00, 9751.33it/s] 
7063it [00:00, 10053.24it/s]


(7063, 225) (7063, 225) 0.04289961772617868 0.04289961772617868
F1: 0.1361573373676248, ACC: 0.919156165935155, AUC: 0.5488334114476536
F1: 0.13458528951486698, ACC: 0.9217046580773043, AUC: 0.5487955748237546
F1: 0.14093959731543623, ACC: 0.927509556845533, AUC: 0.551232741617357
F1: 0.13771517996870108, ACC: 0.9219878238708764, AUC: 0.5524408284023669
Processing margin 7 city Atlantic City  fold -1 ... 


...

6912it [00:00, 7699.22it/s]
6912it [00:00, 8003.20it/s]


(6912, 1225) (6912, 1225) 0.06336805555555555 0.06336805555555555
F1: 0.14191852825229961, ACC: 0.9055266203703703, AUC: 0.5389323362998888
F1: 0.12933753943217666, ACC: 0.9201388888888888, AUC: 0.534349904006613
F1: 0.13333333333333333, ACC: 0.9228877314814815, AUC: 0.5380365861055744
F1: 0.13515825491873396, ACC: 0.8537326388888888, AUC: 0.5429265357883941
Processing margin 20 city Washington  fold -1 ... 


6912it [00:00, 7262.69it/s]
6912it [00:00, 7798.85it/s]


(6912, 2025) (6912, 2025) 0.06336805555555555 0.06336805555555555
F1: 0.15484569572279372, ACC: 0.7741608796296297, AUC: 0.5719086391227008
F1: 0.15067519545131486, ACC: 0.8271122685185185, AUC: 0.560853001045277
F1: 0.1468298109010011, ACC: 0.8890335648148148, AUC: 0.5576469911962567
F1: 0.15226824457593688, ACC: 0.6890914351851852, AUC: 0.5845341323142942


12

In [16]:
dict_meta = {}

f1_full = []
for city in locations:
    for meta_run in dict_models[city]:
        best_f1 = meta_run["params"]["best_f1"]
        
        # print( city, margin, fold, "=>", best_threshold, best_f1)

        if (city not in dict_meta) or (best_f1 > dict_meta[city]["params"]["best_f1"]):
            dict_meta[city] = {
                "model": meta_run["model"],
                "params": meta_run["params"]
            }
                
for city in locations:
    print(dict_meta[city]["params"])

{'fold': -1, 'margin': 15, 'max_samples': 500, 'best_threshold': 0.277, 'acc': 0.9197224975222993, 'auc': 0.5806872107329076, 'best_f1': 0.16}
{'fold': -1, 'margin': 20, 'max_samples': 3000, 'best_threshold': 0.373, 'acc': 0.9490804282185012, 'auc': 0.6156715417264977, 'best_f1': 0.12293144208037825}
{'fold': -1, 'margin': 15, 'max_samples': 1000, 'best_threshold': 0.0, 'acc': 0.06159674477339694, 'auc': 0.4931967703349282, 'best_f1': 0.11604546656093048}
{'fold': -1, 'margin': 10, 'max_samples': 500, 'best_threshold': 0.624, 'acc': 0.9840264389975213, 'auc': 0.5515532552179462, 'best_f1': 0.13432835820895522}
{'fold': -1, 'margin': 3, 'max_samples': 100, 'best_threshold': 0.298, 'acc': 0.9326089946362262, 'auc': 0.5676461076616711, 'best_f1': 0.17229729729729729}
{'fold': -1, 'margin': 5, 'max_samples': 500, 'best_threshold': 0.8270000000000001, 'acc': 0.994060773480663, 'auc': 0.5446753386224209, 'best_f1': 0.12244897959183673}
{'fold': -1, 'margin': 15, 'max_samples': 100, 'best_thr

In [17]:
dict_meta

{'Atlantic City': {'model': <Isolation.IsolationModel at 0x77db4182f310>,
  'params': {'fold': -1,
   'margin': 15,
   'max_samples': 500,
   'best_threshold': 0.277,
   'acc': 0.9197224975222993,
   'auc': 0.5806872107329076,
   'best_f1': 0.16}},
 'Baltimore': {'model': <Isolation.IsolationModel at 0x77db3f19ed30>,
  'params': {'fold': -1,
   'margin': 20,
   'max_samples': 3000,
   'best_threshold': 0.373,
   'acc': 0.9490804282185012,
   'auc': 0.6156715417264977,
   'best_f1': 0.12293144208037825}},
 'Eastport': {'model': <Isolation.IsolationModel at 0x77db3ee85490>,
  'params': {'fold': -1,
   'margin': 15,
   'max_samples': 1000,
   'best_threshold': 0.0,
   'acc': 0.06159674477339694,
   'auc': 0.4931967703349282,
   'best_f1': 0.11604546656093048}},
 'Fort Pulaski': {'model': <Isolation.IsolationModel at 0x77db3cbb3310>,
  'params': {'fold': -1,
   'margin': 10,
   'max_samples': 500,
   'best_threshold': 0.624,
   'acc': 0.9840264389975213,
   'auc': 0.5515532552179462,
   'b

In [18]:
sub_dir = "subs/sub_isolation"
!mkdir -p $sub_dir
with open(f'{sub_dir}/dict_meta.pickle', 'wb') as handle:
    pickle.dump(dict_meta, handle)

In [19]:
model.predict_proba([img.flatten()])

array([[1., 0.]])

In [20]:
prob = model.predict_proba([img.flatten()])
prob

array([[1., 0.]])

In [21]:
1-prob
# array([[0.99483734, 0.00516266]])

array([[0., 1.]])

In [22]:
df_full[df_full["anomaly"] == 1].sort_values("fold")

,t,anomaly,location,latitude,longitude,fold,year
10,1993-01-11,1,Fort Pulaski,32.036700,-80.901700,0,1993
47155,1998-02-27,1,Portland,43.658060,-70.244170,0,1998
48921,2003-01-04,1,Portland,43.658060,-70.244170,0,2003
49261,2003-12-24,1,Portland,43.658060,-70.244170,0,2003
49262,2003-12-25,1,Portland,43.658060,-70.244170,0,2003
...,...,...,...,...,...,...,...
50470,2007-04-16,1,Portland,43.658060,-70.244170,4,2007
48862,2002-11-06,1,Portland,43.658060,-70.244170,4,2002
48671,2002-04-29,1,Portland,43.658060,-70.244170,4,2002
46747,1997-01-11,1,Portland,43.658060,-70.244170,4,1997
